![alt text](./Cerny_logo_1.jpg)

# The effect of ambulance acceleration on mechanical ventilation during neonatal transport

#### Author: Dr Gusztav Belteki

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import scipy.signal

import os
import sys
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'
matplotlib.rcParams['date.autoformatter.second'] = '%H:%M:%S.%f'

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'accelerometer_ventilated'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = os.path.join('/Users', 'guszti', 'ventilation_fabian')

DIR_WRITE = os.path.join(CWD, 'Analyses', TOPIC)

DATA_DUMP = os.path.join('/Volumes', DRIVE, 'data_dump/', 'fabian')

In [ ]:
DIR_WRITE, DATA_DUMP

### 3. Import ventilator and accelerometer data from pickle archives

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_accelero'), 'rb') as handle:
    data_pars_measurements = pickle.load(handle)
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_accelero'), 'rb') as handle:
    data_pars_settings = pickle.load(handle)  

In [ ]:
%%time
with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_1'), 'rb') as handle:
    accelero_ventilated = pickle.load(handle)

In [ ]:
len(data_pars_measurements), len(data_pars_settings), len(accelero_ventilated)

### 4. Calculate recording durations

In [ ]:
recording_durations_vent = {}

for recording in data_pars_measurements:
    if len(data_pars_measurements[recording]) > 0:
         recording_durations_vent[recording] = data_pars_measurements[recording].index[-1] - \
            data_pars_measurements[recording].index[0]
    
recording_durations_vent = Series(recording_durations_vent)
recording_durations_vent;

recording_durations_accel = {}

for recording in accelero_ventilated:
    if len(accelero_ventilated[recording]) > 0:
         recording_durations_accel[recording] = accelero_ventilated[recording].index[-1] - \
            accelero_ventilated[recording].index[0]
    
recording_durations_accel = Series(recording_durations_accel)
recording_durations_accel;

# Combine recording_durations
recording_durations = DataFrame([recording_durations_accel, recording_durations_vent]).T
recording_durations.columns = ['accel', 'vent']

#recording_durations.head()

In [ ]:
recording_durations.info()

In [ ]:
recording_durations.describe()

In [ ]:
recording_durations.sum()

### 5. Remove those recordings that are less that 15 minutes long

Some journey were done within Budapest and hence are very short.
This is not the net time the ambulance was moving. It includes time when the team was moving the trolley within the hospital.

In [ ]:
len(recording_durations)

In [ ]:
len(recording_durations[recording_durations['vent'] >= pd.to_timedelta('15M')])

In [ ]:
recording_durations[recording_durations['vent'] < pd.to_timedelta('15M')]

In [ ]:
len(recording_durations[recording_durations['accel'] >= pd.to_timedelta('15M')])

In [ ]:
recording_durations[recording_durations['accel'] < pd.to_timedelta('15M')]

In [ ]:
matches_to_keep = sorted(recording_durations[recording_durations['accel'] > 
                        pd.to_timedelta('15M')]['accel'].to_dict())
matches_to_keep;

In [ ]:
accelero_ventilated = {key: value for key, value in accelero_ventilated.items() 
                       if key in matches_to_keep}

data_pars_measurements = {key: value for key, value in data_pars_measurements.items() 
                       if key in matches_to_keep}

data_pars_settings = {key: value for key, value in data_pars_settings.items() 
                       if key in matches_to_keep}

In [ ]:
len(data_pars_measurements), len(data_pars_settings), len(accelero_ventilated)

### 6. Import data about times when the team left the Unit and when the ambulance was moving

This table was generated by the clinicians after manually reviewing the case records

The table also contains the ID of the ambulance. Suspension in different ambulances may be different.

In [ ]:
amb_movement_times = pd.read_excel(os.path.join(CWD, 'amb_movement_times_extended.xlsx'), index_col = 'Recording')
amb_movement_times = amb_movement_times[['Time ambulance departed from referring hospital',
                                         'Time ambulance arrived at destination hospital']]
amb_movement_times['Journey time'] = amb_movement_times['Time ambulance arrived at destination hospital'] - \
    amb_movement_times['Time ambulance departed from referring hospital']
amb_movement_times.dropna(inplace = True)

In [ ]:
amb_movement_times.info()

In [ ]:
# In some cases the ambulance journey was very short, less than 10 minutes
len(amb_movement_times[amb_movement_times['Journey time'] < pd.to_timedelta('10M')])

### 7. Keep only periods from the departure of the ambulance to the arrival of the ambulance

There could still be periods when the ambulance stopped during the transfer, these need to be reviewed later.

In [ ]:
rec_list = list(accelero_ventilated.keys())

for recording in rec_list:
    if recording[1] in amb_movement_times.index:
    
        a = amb_movement_times.loc[recording[1]]['Time ambulance departed from referring hospital']
        b = amb_movement_times.loc[recording[1]]['Time ambulance arrived at destination hospital']
    
        accelero_ventilated[recording] = accelero_ventilated[recording][a : b]
        data_pars_measurements[recording] = data_pars_measurements[recording][a : b]
        data_pars_settings[recording] = data_pars_settings[recording][a : b]
    
    else: 
        del accelero_ventilated[recording]
        del data_pars_measurements[recording]
        del data_pars_settings[recording]

In [ ]:
len(data_pars_measurements), len(data_pars_settings), len(accelero_ventilated)

### 8. Calculate recording durations of the trimmed dataset

In [ ]:
recording_durations_vent = {}

for recording in data_pars_measurements:
    if len(data_pars_measurements[recording]) > 0:
         recording_durations_vent[recording] = data_pars_measurements[recording].index[-1] - \
            data_pars_measurements[recording].index[0]
    
recording_durations_vent = Series(recording_durations_vent)
recording_durations_vent;

recording_durations_accel = {}

for recording in accelero_ventilated:
    if len(accelero_ventilated[recording]) > 0:
         recording_durations_accel[recording] = accelero_ventilated[recording].index[-1] - \
            accelero_ventilated[recording].index[0]
    
recording_durations_accel = Series(recording_durations_accel)
recording_durations_accel;

# Combine recording_durations
recording_durations = DataFrame([recording_durations_accel, recording_durations_vent]).T
recording_durations.columns = ['accel', 'vent']

#recording_durations.head()

In [ ]:
recording_durations.info()

In [ ]:
recording_durations.sum()

### 9. Remove those recordings when the net moving time was less than 10 minutes

This still includes potential stops during the journey, they need to be reviewed later

In [ ]:
len(recording_durations)

In [ ]:
len(recording_durations[recording_durations['accel'] > pd.to_timedelta('10M')])

In [ ]:
matches_to_keep = sorted(recording_durations[recording_durations['accel'] > 
                        pd.to_timedelta('10M')]['accel'].to_dict())
matches_to_keep;

In [ ]:
accelero_ventilated = {key: value for key, value in accelero_ventilated.items() 
                       if key in matches_to_keep}

data_pars_measurements = {key: value for key, value in data_pars_measurements.items() 
                       if key in matches_to_keep}

data_pars_settings = {key: value for key, value in data_pars_settings.items() 
                       if key in matches_to_keep}

In [ ]:
len(data_pars_measurements), len(data_pars_settings), len(accelero_ventilated)

### 10. Calculate final recording durations

In [ ]:
recording_durations_vent = {}

for recording in data_pars_measurements:
    if len(data_pars_measurements[recording]) > 0:
         recording_durations_vent[recording] = data_pars_measurements[recording].index[-1] - \
                                     data_pars_measurements[recording].index[0]
    
recording_durations_vent = Series(recording_durations_vent)
recording_durations_vent;

recording_durations_accel = {}

for recording in accelero_ventilated:
    if len(accelero_ventilated[recording]) > 0:
         recording_durations_accel[recording] = accelero_ventilated[recording].index[-1] - \
                                                  accelero_ventilated[recording].index[0]
    
recording_durations_accel = Series(recording_durations_accel)
recording_durations_accel;

# Combine recording_durations
recording_durations = DataFrame([recording_durations_accel, recording_durations_vent]).T
recording_durations.columns = ['accel', 'vent']

#recording_durations.head()

In [ ]:
recording_durations.info()

In [ ]:
recording_durations.sum()

### 11. Subtract gravity from vertical (`Z`) acceleration

In [ ]:
for rec in accelero_ventilated:
    accelero_ventilated[rec]['Z'] = accelero_ventilated[rec]['Z'] - 9.81

### 12. Pass accelerometer data through a low pass filter and a high pass filter

Low pass filter isolates true acceleration, high pass filter isolates vibration

In [ ]:
%%time

# third order Butterworth filter
# Wn is the critical frequency, as fraction of the Nyquist frequency
# As sampling rate was 100 Hz, Nyquist frequency is 50Hz critical frequency is 0.5 Hz
b, a = scipy.signal.butter(N=3, Wn=0.01, btype='lowpass')
c, d = scipy.signal.butter(N=3, Wn=0.01, btype='highpass')

accelero_ventilated_low_pass = {}
accelero_ventilated_high_pass = {}

for recording, data in accelero_ventilated.items():
    #print(recording)
    data_low_pass = {}
    data_high_pass = {}
    for item in ['X', 'Y', 'Z']:
        data_low_pass[item] =  scipy.signal.filtfilt(b, a, data[item])
        data_high_pass[item] =  scipy.signal.filtfilt(c, d, data[item])
    
    accelero_ventilated_low_pass[recording] = DataFrame(data_low_pass)
    accelero_ventilated_low_pass[recording].index = accelero_ventilated[recording].index
    accelero_ventilated_high_pass[recording] = DataFrame(data_high_pass)
    accelero_ventilated_high_pass[recording].index = accelero_ventilated[recording].index

### 13. Calculate absolute values of the accelerations in each direction as well as the length (Euclidean norm, also known as L2 norm) of the acceleration vector

In [ ]:
%%time

for rec in accelero_ventilated:
    
    # absolute value of the vectors
    accelero_ventilated[rec]['X_abs'] = np.abs(accelero_ventilated[rec]['X'])
    accelero_ventilated[rec]['Y_abs'] = np.abs(accelero_ventilated[rec]['Y'])
    accelero_ventilated[rec]['Z_abs'] = np.abs(accelero_ventilated[rec]['Z'])
    
    # Euclidean norm of the acceleration vector
    accelero_ventilated[rec]['length'] = np.sqrt(accelero_ventilated[rec]['X'] ** 2 + 
        accelero_ventilated[rec]['Y'] ** 2 + accelero_ventilated[rec]['Z'] ** 2)

In [ ]:
%%time

for rec in accelero_ventilated_low_pass:
    
    # absolute value of the vectors
    accelero_ventilated_low_pass[rec]['X_abs'] = np.abs(accelero_ventilated_low_pass[rec]['X'])
    accelero_ventilated_low_pass[rec]['Y_abs'] = np.abs(accelero_ventilated_low_pass[rec]['Y'])
    accelero_ventilated_low_pass[rec]['Z_abs'] = np.abs(accelero_ventilated_low_pass[rec]['Z'])
    
    # Euclidean norm of the acceleration vector
    accelero_ventilated_low_pass[rec]['length'] = np.sqrt(accelero_ventilated_low_pass[rec]['X'] ** 2 + 
        accelero_ventilated_low_pass[rec]['Y'] ** 2 + accelero_ventilated_low_pass[rec]['Z'] ** 2)

In [ ]:
%%time

for rec in accelero_ventilated_high_pass:
    
    # absolute value of the vectors
    accelero_ventilated_high_pass[rec]['X_abs'] = np.abs(accelero_ventilated_high_pass[rec]['X'])
    accelero_ventilated_high_pass[rec]['Y_abs'] = np.abs(accelero_ventilated_high_pass[rec]['Y'])
    accelero_ventilated_high_pass[rec]['Z_abs'] = np.abs(accelero_ventilated_high_pass[rec]['Z'])
    
    # Euclidean norm of the acceleration vector
    accelero_ventilated_high_pass[rec]['length'] = np.sqrt(accelero_ventilated_high_pass[rec]['X'] ** 2 + 
        accelero_ventilated_high_pass[rec]['Y'] ** 2 + accelero_ventilated_high_pass[rec]['Z'] ** 2)

### 14. Calculate the directional X and Y acceleration (separate positive and negative values in these directions)

In [ ]:
%%time

for rec in accelero_ventilated_low_pass:
    
    # generate columns with only the positive or only the negative values of X and Y acceleleration considered
    accelero_ventilated_low_pass[rec]['X_pos'] = accelero_ventilated_low_pass[rec]['X'].apply(lambda x: max(x, 0))
    accelero_ventilated_low_pass[rec]['X_neg'] = accelero_ventilated_low_pass[rec]['X'].apply(lambda x: min(x, 0))
    accelero_ventilated_low_pass[rec]['Y_pos'] = accelero_ventilated_low_pass[rec]['Y'].apply(lambda x: max(x, 0))
    accelero_ventilated_low_pass[rec]['Y_neg'] = accelero_ventilated_low_pass[rec]['Y'].apply(lambda x: min(x, 0))

### 15. Resample accelerometer data

Accelerometer data are not following a normal distribution. Use median.

In [ ]:
%%time

accelero_ventilated_1min_median = {}

columns_to_keep = ['X_abs', 'Y_abs', 'Z_abs', 'length']

for rec in sorted(accelero_ventilated.keys()):
    accelero_ventilated_1min_median[rec] = accelero_ventilated[rec].resample('1Min').median()[columns_to_keep]

In [ ]:
%%time

accelero_ventilated_low_pass_1min_median = {}

columns_to_keep_1 = ['X_abs', 'Y_abs', 'Z_abs', 'length']
columns_to_keep_2 = ['X_pos', 'X_neg', 'Y_pos', 'Y_neg']

for rec in sorted(accelero_ventilated_low_pass.keys()):
    
    accelero_ventilated_low_pass_1min_median[rec] = \
        accelero_ventilated_low_pass[rec].resample('1Min').median()[columns_to_keep_1]
    
    # Az 0 data points here represent negative acceleration values converted to zeros
    # they should be excluded from median
    for par in columns_to_keep_2:
        data = accelero_ventilated_low_pass[rec][par][accelero_ventilated_low_pass[rec][par] != 0]
        accelero_ventilated_low_pass_1min_median[rec][par] = data.resample('1Min').median()
    
    accelero_ventilated_low_pass_1min_median[rec].rename(lambda x: x + '_median', axis=1, inplace=True)

In [ ]:
%%time

accelero_ventilated_high_pass_1min_median = {}

columns_to_keep = ['X_abs', 'Y_abs', 'Z_abs', 'length']

for rec in sorted(accelero_ventilated_low_pass.keys()):
    
    accelero_ventilated_high_pass_1min_median[rec] = \
        accelero_ventilated_high_pass[rec].resample('1Min').median()[columns_to_keep]
    
    accelero_ventilated_high_pass_1min_median[rec].rename(lambda x: x + '_median', axis=1, inplace=True)

### 16. Keep only relevant columns of ventilator data

In [ ]:
for recording in data_pars_measurements:
    
    # MV and VT are not meaningful without weight correction
    data_pars_measurements[recording].drop(['MV', 'VTemand', 'VTimand'], axis = 1, inplace = True)
    
    if 'VTemand_resp' in data_pars_measurements[recording]:
        # Only SIMV recordings have these columns
        data_pars_measurements[recording].drop(['VTemand_resp', 'VTespon_pat'], axis = 1, inplace = True)

    # Remove empty rows
    data_pars_measurements[recording].dropna(how = 'all', axis = 1, inplace = True)

### 17. Add relevant columns from ventilator settings

In [ ]:
setting_to_keep_1 = ['PIP_set', 'PEEP_set', 'FiO2_set', 'Ti_set', 'Te_set', 'RR_set', 'VG_set_kg', 
                     'IE_I_set', 'IE_E_set', 'Trigger_sens_set', 'Ventilator_mode']

setting_to_keep_1B = ['PIP_set', 'PEEP_set', 'FiO2_set', 'Ti_set', 'Te_set', 'RR_set', 
                      'IE_I_set', 'IE_E_set', 'Trigger_sens_set', 'Ventilator_mode']

data_pars_combined = {}

for recording in data_pars_measurements:
    # Recordings containing volume guarantee ventilation
    if 'VG_set_kg' in data_pars_settings[recording]:
        data_pars_combined[recording] = pd.merge(data_pars_measurements[recording], 
                data_pars_settings[recording][setting_to_keep_1], left_index= True, right_index = True)
    
    # Recordings without VG
    else: 
        data_pars_combined[recording] = pd.merge(data_pars_measurements[recording], 
                data_pars_settings[recording][setting_to_keep_1B], left_index= True, right_index = True)

### 18. Add derived parameters to ventilator data

- VTdiff: the difference between the sew and the actual VT during VG ventilation
- Pdiff_VG: the difference between Pmax and PIP during VG ventilation
- Pdiff_noVG: the difference between the set and the actual PIP during pressure limited ventilation
- RR_diff: the difference between actual RR and the set RR (this will only work during SIPPV as during SIMV the spontaneous breaths' RR is not reported.

In [ ]:
for recording in data_pars_combined:
    if 'VG_set_kg' in data_pars_combined[recording]:
        data_pars_combined[recording]['VTdiff'] = (data_pars_combined[recording]['VTemand_kg'] - 
                                                   data_pars_combined[recording]['VG_set_kg'])
        data_pars_combined[recording]['VTdiff_abs'] = np.abs(data_pars_combined[recording]['VTemand_kg'] - 
                                                             data_pars_combined[recording]['VG_set_kg'])
        data_pars_combined[recording]['Pdiff_VG'] = (data_pars_combined[recording]['PIP_set'] - 
                                                     data_pars_combined[recording]['PIP'])
        data_pars_combined[recording]['Pdiff_VG_abs'] = np.abs(data_pars_combined[recording]['PIP_set'] - 
                                                               data_pars_combined[recording]['PIP'])
                                                     
    else:
        data_pars_combined[recording]['Pdiff_noVG'] = (data_pars_combined[recording]['PIP_set'] - 
                                                       data_pars_combined[recording]['PIP'])
        data_pars_combined[recording]['Pdiff_noVG_abs'] = np.abs(data_pars_combined[recording]['PIP_set'] - 
                                                                 data_pars_combined[recording]['PIP'])

In [ ]:
for recording in data_pars_combined:
    if 'RR' in data_pars_combined[recording] and 'RR_set' in data_pars_combined[recording]:
        data_pars_combined[recording]['RRdiff'] = (data_pars_combined[recording]['RR'] - 
                                                   data_pars_combined[recording]['RR_set'])
        data_pars_combined[recording]['RRdiff_abs'] = np.abs(data_pars_combined[recording]['RR'] - 
                                                             data_pars_combined[recording]['RR_set'])

### 19. Generate DataFrames with the ventilator modes only

In [ ]:
ventilator_mode = {}

for recording in data_pars_settings:
    ventilator_mode[recording] = data_pars_settings[recording][['Ventilator_mode']] 

ventilator_mode[('default__393.txt', 'AL000628')];

### 20. Resample ventilator data

Some ventilator parameters follow normal distribution. For them, use mean and SD for aggregation.
Other parameters do not follow normal distribution. For them, calculate median and IQR.

In [ ]:
%%time

data_pars_combined_1min_mean = {}
data_pars_combined_1min_std = {}
data_pars_combined_1min_median = {}

for recording in data_pars_combined:
    
    data_pars_combined_1min_mean[recording] = data_pars_combined[recording].resample('1Min').mean()
    data_pars_combined_1min_std[recording] = data_pars_combined[recording].resample('1Min').std()
    data_pars_combined_1min_median[recording] = data_pars_combined[recording].resample('1Min').median()
    data_pars_combined_1min_median[recording].rename(mapper = lambda x: x + '_median', axis = 1, inplace = True)

### 21. Combine aggregated accelerometer and ventilator data

In [ ]:
combined_ventilated_1min  = {}

for recording in accelero_ventilated:
    
    combined_ventilated_1min[recording] = pd.merge(data_pars_combined_1min_mean[recording], 
        data_pars_combined_1min_std[recording],
        how = 'inner', left_index = True, right_index= True, suffixes = ['_mean', '_sd'])
    
    combined_ventilated_1min[recording] = pd.merge(combined_ventilated_1min[recording], 
        data_pars_combined_1min_median[recording],
        how = 'inner', left_index = True, right_index= True,)
    
    combined_ventilated_1min[recording] = pd.merge(combined_ventilated_1min[recording], 
        accelero_ventilated_low_pass_1min_median[recording],
        how = 'inner', left_index = True, right_index= True)
    
    combined_ventilated_1min[recording] = pd.merge(combined_ventilated_1min[recording], 
        accelero_ventilated_high_pass_1min_median[recording], 
        how = 'inner', left_index = True, right_index= True, suffixes = ['_low_pass', '_high_pass'])
    
    combined_ventilated_1min[recording].dropna(how = 'all', axis = 1, inplace = True)

### 22. Export aggregated accelerometer data as pickle archives

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_combined'), 'wb') as handle:
    pickle.dump(data_pars_combined, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('%s/%s.pickle' % (DATA_DUMP, 'combined_ventilated_1min'), 'wb') as handle:
    pickle.dump(combined_ventilated_1min, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [ ]:
%%time

with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_2'), 'wb') as handle:
    pickle.dump(accelero_ventilated, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_low_pass'), 'wb') as handle:
    pickle.dump(accelero_ventilated_low_pass, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_high_pass'), 'wb') as handle:
    pickle.dump(accelero_ventilated_high_pass, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'ventilator_mode'), 'wb') as handle:
    pickle.dump(ventilator_mode, handle, protocol=pickle.HIGHEST_PROTOCOL)